### import libraries

In [1]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


In [30]:
numpy.random.seed(7)

### import the dataset (each of the three types and select the required columns)

In [31]:
#df = pandas.read_csv('avgbirdscountbyidwithdate.csv', usecols=[0,2,4], engine='python', skipfooter=3)

#dataframe = df[df["ID"] == 120]
#dataframe = dataframe.drop('ID', 1)
#dataframe = dataframe[1:]
#print dataframe
#dataframe = dataframe["latitude":"latitude"]
dataframe = pandas.read_csv('AvgbirdDataWithWeather.csv', usecols=[0,2,4,5], engine='python', skipfooter=3)
dataframe = dataframe[1:]
dataset = dataframe.values
print dataset
dataset = dataset.astype('float32')


[[ 208.           38.7803       25.385     ]
 [ 209.           38.78033077   26.03      ]
 [ 210.           38.78035769   25.135     ]
 ..., 
 [ 325.            9.1481425     8.13      ]
 [ 326.            8.82693465    7.56      ]
 [ 327.            9.03829324    8.79      ]]


### Normalise the input data 

In [71]:
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

### Split into test and training dataset

In [32]:
train_size = int(len(dataset) * 0.70)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

(2602, 778)


### Create the new dataset from the existing dataset (t -> t+1) lookback =1

In [33]:
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0:3]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 1])
	return numpy.array(dataX), numpy.array(dataY)

In [34]:
x,y = create_dataset(dataset)

In [35]:
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

### Reshape it in case where the input dimension is more than 1.

In [63]:
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

ValueError: total size of new array must be unchanged

### Define the model

In [36]:
def smallmodel():
    model = Sequential()
    model.add(LSTM(4, input_dim=3))
    #model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [37]:
model = smallmodel()
#model.load_weights("bird_120_model_weights_withday.h5",by_name=False)
model.fit(trainX, trainY, nb_epoch=30, batch_size=1, verbose=2)
#model.save("bird_389_model_weights_withday_modified2.h5")

Epoch 1/30
18s - loss: 550.3670
Epoch 2/30
17s - loss: 413.0591
Epoch 3/30
18s - loss: 278.6007
Epoch 4/30
15s - loss: 203.6421
Epoch 5/30
15s - loss: 148.6804
Epoch 6/30
15s - loss: 106.9117
Epoch 7/30
15s - loss: 66.0822
Epoch 8/30
15s - loss: 39.4953
Epoch 9/30
15s - loss: 24.3931
Epoch 10/30
15s - loss: 16.0941
Epoch 11/30
15s - loss: 11.9030
Epoch 12/30
15s - loss: 9.6819
Epoch 13/30
15s - loss: 8.8471
Epoch 14/30
15s - loss: 7.9556
Epoch 15/30
15s - loss: 7.6823
Epoch 16/30
15s - loss: 7.4238
Epoch 17/30
15s - loss: 6.9263
Epoch 18/30
15s - loss: 6.8177
Epoch 19/30
15s - loss: 6.9147
Epoch 20/30
15s - loss: 6.3368
Epoch 21/30
15s - loss: 6.1355
Epoch 22/30
15s - loss: 6.4396
Epoch 23/30
15s - loss: 6.0066
Epoch 24/30
15s - loss: 5.9780
Epoch 25/30
15s - loss: 5.9750
Epoch 26/30
15s - loss: 6.4793
Epoch 27/30
15s - loss: 6.0387
Epoch 28/30
15s - loss: 6.1736
Epoch 29/30
16s - loss: 5.8923
Epoch 30/30
15s - loss: 5.7363


### Phase 1 : Predictions on test and training set

In [17]:

# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
#trainPredict = scaler.inverse_transform(trainPredict)
#trainY = scaler.inverse_transform([trainY])
#testPredict = scaler.inverse_transform(testPredict)
#testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY, trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY, testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))


Train Score: 2.41 RMSE
Test Score: 2.09 RMSE


### Phase 2 : prediction, Re determine the complete dataset through LSTM.

In [38]:
trainPredict =[]
t2 = dataset[0][1]

trainPredict.append([[t2.tolist()]])
#print trainPredict
for i in range(3378):
    l=[]
    l.append([[dataset[i][0],t2,dataset[i][2]]])
    #print l
    l2 = numpy.asarray(l)
    t2 = model.predict(l2)
    #print t2.tolist()
    trainPredict.append(t2.tolist())

### Plot the graphs and the RMSE values

In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from datetime import datetime 
import matplotlib.dates as md
df1 = pandas.read_csv('AvgbirdDataWithWeather.csv', usecols=[1], engine='python', skipfooter=3)
#df1 = pandas.read_csv('avgbirdscountbyidwithdate.csv', usecols=[1,4], engine='python', skipfooter=3)

#dataframe = df1[df1["ID"] == 120]
#dataframe = dataframe.drop('ID', 1)
#dataframe = dataframe[1:]
#df1 = dataframe

ax=plt.gca()
xfmt = md.DateFormatter('%Y-%m-%d')
ax.xaxis.set_major_formatter(xfmt)

tme = df1[1:].values.tolist()
x = []
#print now.strftime("%Y-%m-%d %H:%M:%S")
for i in tme:
    x.append([datetime.strptime(d,"%Y-%m-%d") for d in i])
print len(x)

plt.xticks( rotation=25 )
plt.xlabel("Date")
plt.ylabel("latitude")
y_axis_predicted=[]
y_axis_actual=[]
y_axis_actual_test=[]
x_axis=[]
#print trainPredict[100][0][0]
for i in range(3378):
    if i==0:
        continue
    x_axis.append([datetime.strptime(d,"%Y-%m-%d") for d in tme[i]])
    #print trainPredict[i][0][0]
    y_axis_predicted.append(trainPredict[i][0][0])
    y_axis_actual.append(dataset[i][1])
    #y_axis_actual.append(trainX[i][0][1])
    #y_axis_actual.append(testX[i][0][1])
trainScore = math.sqrt(mean_squared_error(y_axis_actual, y_axis_predicted))
print('Train Score: %.2f RMSE' % (trainScore))
plt.plot(x_axis,y_axis_predicted,color = 'r',label = 'Predicted using lstm')
plt.plot(x_axis,y_axis_actual,color= 'g',label = 'Original dataset')
plt.legend(loc="lower right")
plt.ylim([-15, 50])
plt.show()

NameError: name 'pandas' is not defined

In [17]:
from datetime import datetime 
import matplotlib.dates as md

In [2]:
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
#plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

NameError: name 'numpy' is not defined